In [3]:
import datetime as dt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_table("dataset_521000_9.txt", delimiter=";")  # Читаем файл
df['date'] = pd.to_datetime(df['date'])  # Переводим столбец даты дату
df.sort_values(by=['seller_id', 'date'], inplace=True)  # Отсортируем по магазинам и дате

df['period'] = df['date'].dt.date.diff(periods=1).apply(lambda x: x.days)  # Считаем разницу между соседними продажами
df.reset_index(drop=True, inplace=True)

df['period'][0] = 1  # Зададим период для нулевой строки
df['rank_period'] = 1  # Зададим ранк для нулевой стркои

'''
Не придумал как сделать это без цикла.
Как можно в следующей строке использовать только что измененное значение предыдущей (???)
'''
for i in range(1, df.shape[0]):
    df['rank_period'][i] = df['rank_period'][i - 1] if df['period'][i] == 1.0 and df['seller_id'][i] == df['seller_id'][i - 1] \
                        else (df['rank_period'][i - 1] + 1)
    

# Далее считаем все требуемые величины
len_period = list(df.groupby('rank_period').count()['period'])
mean_period_length = round(sum(len_period) / len(len_period), 3)

rank_period_quant_array = list(df.groupby('rank_period').sum()['quantity'] / len_period)
mean_sales_count = round(sum(rank_period_quant_array) / len(rank_period_quant_array), 3)

mean_periods_count = round(df.groupby('seller_id').nunique()['rank_period'].sum() / df.seller_id.nunique(0), 3)

print(f'mean_period_length:{mean_period_length} mean_sales_count:{mean_sales_count} mean_periods_count:{mean_periods_count}')

mean_period_length:4.848 mean_sales_count:19.738 mean_periods_count:4.253
